In [1]:
import math
import numpy as np
import csv
import skimage
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import pathlib

In [2]:
def draw_circle(img, row: int, col: int, rad: int):
    """
    Draw a circle directly into the Numpy Array
    """
    rr, cc, val = skimage.draw.circle_perimeter_aa(row, col, rad, shape=img.shape)
    valid = (
            (rr >= 0) &
            (rr < img.shape[0]) &
            (cc >= 0) &
            (cc < img.shape[1])
    )
    # skimage.draw.set_color(processed_img, (rr[valid], cc[valid]), [255, 255, 255], alpha=val[valid])
    if len(img.shape) == 2:
        # Black and White Image
        val = np.multiply(val, 255)
        img[rr[valid], cc[valid]] = val[valid]
    elif len(img.shape) == 3:
        # RGB Image
        skimage.draw.set_color(img, (rr[valid], cc[valid]), [255, 255, 255], alpha=val[valid])

In [3]:
def generate_circle(height: int, width: int, min_radius: int, max_radius: int, totally_inside=False):
    """

    :param height:
    :param width:
    :param min_radius:
    :param max_radius:
    :param totally_inside:
    :return:
    """
    # Initialize parameters
    center_x, center_y, rad = math.inf, math.inf, math.inf
    #
    if totally_inside:
        while center_x - rad < 0 or center_x + rad > width or center_y - rad < 0 or center_y + rad > height:
            # Get radius
            rad = np.random.randint(min_radius, max_radius)
            # Get center
            center_x = np.random.randint(width)
            center_y = np.random.randint(height)
            # Validate and fix if possible
            if center_x - rad < 0 or center_x + rad > width:
                center_x = rad + np.random.randint(width - rad)
            if center_y - rad < 0 or center_y + rad > height:
                center_y = rad + np.random.randint(height - rad)
            # NOTE: If the Fix has not worked, then a new radius is obtained and repeat the process
    else:
        center_y = np.random.randint(height)
        center_x = np.random.randint(width)
        rad = np.random.randint(min_radius, max_radius)
    #
    return center_x, center_y, rad

In [4]:
def noisy_circle(height: int, width: int, channels: int, min_radius: int, max_radius: int, noise: float,
                 contained=False):
    """
    Create and image and Ground of Truth for training

    :param height:
    :param width:
    :param channels:
    :param min_radius:
    :param max_radius:
    :param noise:
    :param contained:
    :return:
    """
    # Create an empty (black) square canvas
    img = None
    if channels == 1:
        img = np.zeros((height, width), dtype=np.uint8)
    elif channels == 3:
        img = np.zeros((height, width, channels), dtype=np.uint8)
    # Generate circle parameters
    center_x, center_y, rad = generate_circle(height, width, min_radius, max_radius, totally_inside=contained)
    # Draw circle
    draw_circle(img, center_y, center_x, rad)

    # Add uniform noise
    uniform_noise = np.asarray(255*noise*np.random.rand(*img.shape), dtype=np.uint8)
    img += uniform_noise
    
    # Return data
    return (center_x/width, center_y/height, rad/max(width, height)), img
    # return (center_x, center_y, rad), img

# Main

In [5]:
# Total number of images to be generated
total_images = 5000
# Size of the image
img_height_px = 200
img_width_px = 200
# Channels
channels_dict = {"BW": 1, "RGB": 3}
img_channels = channels_dict["RGB"]
# Radius range
min_radius = 20
max_radius = 80
if min_radius >= max_radius:
    raise ValueError("Minimum Radius must be lower than Maximum Radius")
# Noise
noise = 0.0
# Generated circle totally contained flag
contained = True
# Destination Path
dest_dir = pathlib.Path('./dataset')
if not dest_dir.exists():
    try:
        os.makedirs(dest_dir)
        print("Created directory {}".format(dest_dir))
    except Exception as exc:
        print(f"Exception {exc} creating dir: {dest_dir}")
        exit(-1)

for img_number in tqdm(range(total_images), desc="Generating Dataset", ncols=80):
    # Generate file name
    filename = 'img{:06d}'.format(img_number)
    # Generate circle
    params, img = noisy_circle(img_height_px, img_width_px, img_channels,
                               min_radius, max_radius, noise, contained)
    # Save
    plt.imsave(dest_dir / '{}.jpg'.format(filename), img)
    with open(dest_dir / '{}.txt'.format(filename), 'w') as csvfile:
        spamwriter = csv.writer(csvfile, dialect='excel', delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(['{:0.8f}'.format(params[0]), '{:0.8f}'.format(params[1]), '{:0.8f}'.format(params[2])])


Generating Dataset: 100%|██████████████████| 5000/5000 [00:08<00:00, 612.67it/s]
